In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.cluster import KMeans

<h2>Utilizando os critérios de validação</h2>

In [2]:
# Criando função para realizar a validação dos clusters

def n_cluster_validation(n_clusters_list, base, SEED=20):
    '''
    A função recebe a base de dados normalizada e uma lista contendo os números de clusters a serem testados
    para um modelo k-means e retorna um dataframe contendo os scores para validação
    '''

    resultados = pd.DataFrame(columns=['n clusters', 'Silhouette', 'Davies-Boulding', 'Calinsk-Harabasz'])

    for i in n_clusters_list:
        np.random.seed(SEED)
        predict = KMeans(n_clusters = i, n_init=10).fit_predict(base)
        sc = metrics.silhouette_score(base, predict, metric='euclidean')
        dbs = metrics.davies_bouldin_score(base, predict)
        chs = metrics.calinski_harabasz_score(base, predict)

        resultados = pd.concat([resultados, pd.DataFrame(columns=['n clusters', 'Silhouette', 'Davies-Boulding', 'Calinsk-Harabasz'], data=[[i, sc, dbs, chs]])], axis=0)
    
    return resultados

In [3]:
# Importando os valores normalizados
valores_normalizados = pd.read_csv(r'Dados/valores_normalizados.csv', header=None).values

In [4]:
len(valores_normalizados[0])

16

In [5]:
# Realizando a validação para valores de 2 a 10 clusters
validacao = n_cluster_validation(range(2,11), valores_normalizados)

In [6]:
# Verificando resultados
validacao

,n clusters,Silhouette,Davies-Boulding,Calinsk-Harabasz
0,2,0.299140,1.518087,3321.846358
0,3,0.327220,1.309607,3526.440520
0,4,0.348257,1.221290,3528.717622
0,5,0.364527,1.075856,3431.788780
0,6,0.364854,1.045346,3523.516921
0,7,0.327845,1.150915,3398.388472
0,8,0.308891,1.174854,3239.112230
0,9,0.308185,1.198410,3106.664097
0,10,0.351323,1.117369,3019.117443


Como o modelo com 6 clusters teve o melhor score Silhouette e o melhor score Davies-Boulding, assim como o terceiro melhor score Calinsk-Harabasz, ele será o escolhido.

<h2>Utilizando uma base de dados 'dummy'</h2>

Outra maneira de validar os dados é criando uma base de dados aleatória e comparar com o resultado da base de dados original:

In [7]:
# Criando uma base de dados com valores aleatórios com o mesmo formato da base de dados utilizada

random_data = np.random.rand(len(valores_normalizados),len(valores_normalizados[0]))

# Verificando o desemepenho do algoritmo para essa base de dados:

validacao_dummy = n_cluster_validation([6], random_data)

validacao_dummy

,n clusters,Silhouette,Davies-Boulding,Calinsk-Harabasz
0,6,0.039733,3.338206,279.502946


Como é possível observar, os valores de todos os critérios foram muito piores para a base dummy. Portanto, isso é um bom indicativo de que o modelo real obtido está acima desta baseline.

<h2>Dividindo a base de dados - verificando a estabilidade dos <i>clusters</i></h2>

Outra maneira de validar o modelo é dividindo a base de dados em n partes e verificando se as métricas se mantém estáveis

In [8]:
# Dividindo a base em 3 partes:
set1, set2, set3 = np.array_split(valores_normalizados, 3)

lista_sets = [set1, set2, set3]

# Testando o desempenho de cada parte:
resultados = pd.DataFrame(columns=['set', 'n clusters', 'Silhouette', 'Davies-Boulding', 'Calinsk-Harabasz'])

count = 0

for i in lista_sets:
    count = count + 1
    resultados_intermediarios = n_cluster_validation([6], i)
    resultados_intermediarios = pd.concat([pd.DataFrame(columns=['set'], data=[[count]]), resultados_intermediarios], axis = 1)
    resultados = pd.concat([resultados, resultados_intermediarios], axis = 0)

In [9]:
resultados

,set,n clusters,Silhouette,Davies-Boulding,Calinsk-Harabasz
0,1,6,0.364216,1.022584,1192.388843
0,2,6,0.364943,1.059534,1169.481552
0,3,6,0.372281,1.041536,1184.794637


Todas as métricas tiveram valores próximos em todas as 3 divisões da base de dados, o que indica fortemente que os *clusters* são estáveis.

In [10]:
# Exportando o modelo final
np.random.seed(20)
kmeans = KMeans(n_clusters = 6, n_init=10).fit(valores_normalizados)
predict_modelo_final = kmeans.predict(valores_normalizados)
centroides_finais = kmeans.cluster_centers_

# Exportando os clusters finais
np.savetxt(r'Dados/clusters_modelo_final.csv', predict_modelo_final, delimiter=',')

# Exportando os centróides finais
np.savetxt(r'Dados/centroides_modelo_final.csv', centroides_finais, delimiter=',')